In [20]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir
    path.append(dir(path[0])+'/dash_app')
    __package__ = "dash_app"

In [21]:
import glob, os
import pandas as pd
from settings import *


In [22]:
naming_df = pd.read_csv('../ecv_naming_convention.csv')
naming_df

,chapter,chapt_no,chapt_title,domain,sub_domain,standarised,checked,figure_no,root_variable,variable,available_data,unit,attributes
0,2.1 Surface Air Temperature,2.10,Surface Air Temperature,atmoshperic,sa,y,NaN,2.1.,surface_air_temperature,surface_air_temperature,mean__annual__surface_air_temperature,C,locaion
1,2.1 Surface Air Temperature,2.10,Surface Air Temperature,atmoshperic,sa,y,NaN,2.1.,surface_air_temperature,surface_air_temperature,normal_1961_1990__surface_air_temperature,C,locaion
2,2.1 Surface Air Temperature,2.10,Surface Air Temperature,atmoshperic,sa,y,NaN,2.1.,surface_air_temperature,surface_air_temperature,moving_average__11year__surface_air_temperature,C,locaion
3,2.1 Surface Air Temperature,2.10,Surface Air Temperature,atmoshperic,sa,y,NaN,2.1.,surface_air_temperature,surface_air_temperature_anomaly,mean__annual__surface_air_temperature_anomaly,C,locaion
4,2.1 Surface Air Temperature,2.10,Surface Air Temperature,atmoshperic,sa,y,NaN,2.1.,surface_air_temperature,surface_air_temperature_anomaly,moving_average__11year__surface_air_temperatur...,C,locaion
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,3.3 Ocean Surface and Subsurface Currents,3.30,Ocean Surface and Subsurface Currents,oceanic,ssop,na,NaN,NaN,NaN,NaN,____,NaN,NaN
114,3.8 Nutrients,3.80,Nutrients,oceanic,obgc,NaN,NaN,NaN,NaN,NaN,____,NaN,NaN
115,3.10 Plankton,3.10,Plankton,oceanic,obe,NaN,NaN,NaN,NaN,NaN,____,NaN,NaN
116,3.11 Marine Habitat Properties,3.11,Marine Habitat Properties,oceanic,obe,NaN,NaN,NaN,NaN,NaN,____,NaN,NaN


In [23]:

d = {'domain': [], 'chapter': [], 'figure': [], 'csv_file': [], 'available_data': [], 'unit': [],'other_file':[]}
ignore_files=['Sea_level_IrishStationsMonthlyAverage_for_online_charts.csv',
              'NIR_GHG_Emissions_CSRI2020_forOnline.csv',
             ]
file_struct_df = pd.DataFrame(d)
for filepath in glob.iglob(DATA_PATH+'**', recursive=True):
    if os.path.isfile(filepath): # filter dirs
        file = filepath.split('/')[-1]
        figure = filepath.split('/')[-2]
        chapter = filepath.split('/')[-3]
        domain = filepath.split('/')[-4]
        if ('Map' in figure) | (file in ignore_files):
            continue
#         print(filename)
        if '.csv' in file:
            data_df=pd.read_csv(filepath)
            cols=list(data_df.columns)
            for col in cols:
                if col in ['index','Unnamed: 0']:
                    continue
                try:
                    unit = naming_df.loc[naming_df['available_data'] == col, 'unit'].iloc[0]
                except:
                    unit='('+col+')'
                d = {'domain': domain, 
                     'chapter': chapter, 
                     'figure': figure, 
                     'csv_file': file, 
                     'available_data': col,
                     'unit':unit}
                file_struct_df=file_struct_df.append(d,ignore_index=True)
        elif '.xlsx' in file or '.txt' in file:
            d = {'domain': domain, 
                 'chapter': chapter, 
                 'figure': figure, 
                 'other_file': file}
            file_struct_df=file_struct_df.append(d,ignore_index=True)
#         else:
#             d = {'domain': domain, 
#                  'chapter': chapter, 
#                  'figure': figure, 
#                  'other_file': 'NO_DATA'}
#             if not (figure in file_struct_df['figure']):
#                 file_struct_df=file_struct_df.append(d,ignore_index=True)
            
file_struct_df.sort_values(['chapter','figure'],inplace=True)

In [24]:
file_struct_df.to_csv('../data_files_list.csv',index=False)